In [41]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model, Sequential
import numpy as np

In [42]:
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(",")[0]
        text = ' '.join(lines[i].split(',')[1:])
        text = re.sub(',','', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

df_train = get_dataframe('sentences.txt')
df_train.head()

,label,text
0,0,I need daily rations in Attur
1,0,Give rice dal oil
2,0,people are hungary
3,0,provide rations for 6 people for 10 days
4,0,Need prepared food for 20 people in slum


In [43]:
# df_train

In [44]:
### list of one-hot encoded labels
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [45]:
train_text.shape

(40, 1)

In [46]:
category_counts = len(df_train.label.cat.categories)
module_url = "../tf_sent_encoder_2" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [47]:
embed_size

512

In [48]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

def sentence_encoder(input_text):
    return embed(tf.squeeze(input_text))

# def UniversalEmbedding(x):
#     import tensorflow as tf
#     import tensorflow_hub as hub
#     module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
#     embed = hub.Module(module_url)
# #     embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
#     return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [49]:

input_text = Input(shape=(1,), dtype=tf.string)
# input_text = Input(tf.placeholder(dtype=tf.string, shape=[None]))
#embedding = embed(input_text)
embedding = Lambda(sentence_encoder, output_shape=(embed_size,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
lambda_23 (Lambda)           (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 645       
Total params: 164,869
Trainable params: 164,869
Non-trainable params: 0
_________________________________________________________________


In [51]:
df_test = get_dataframe('test.txt')
### list of one-hot encoded labels
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [52]:
from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=50)
  model.save_weights('./model.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40 samples, validate on 10 samples
Epoch 1/50




40/40 [==============================] - 11s 267ms/step - loss: 1.5980 - acc: 0.4000 - val_loss: 1.5522 - val_acc: 0.6000
Epoch 2/50
40/40 [==============================] - 0s 2ms/step - loss: 1.5172 - acc: 0.9000 - val_loss: 1.5125 - val_acc: 0.6000
Epoch 3/50
40/40 [==============================] - 0s 1ms/step - loss: 1.4505 - acc: 0.9250 - val_loss: 1.4700 - val_acc: 0.7000
Epoch 4/50
40/40 [==============================] - 0s 1ms/step - loss: 1.3749 - acc: 0.9750 - val_loss: 1.4193 - val_acc: 0.7000
Epoch 5/50
40/40 [==============================] - 0s 1ms/step - loss: 1.2937 - acc: 1.0000 - val_loss: 1.3603 - val_acc: 0.7000
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 1.1973 - acc: 1.0000 - val_loss: 1.2933 - val_acc: 0.7000
Epoch 7/50
40/40 [==============================] - 0s 1ms/step - loss: 1

In [54]:
session = tf.Session()
K.set_session(session)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
model.load_weights('./model.h5')  


In [55]:
new_text = ["wheat exhausted in hospital canteen", 
            "I was abused badly"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
predicts = model.predict(new_text)

categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

['1', '4']


In [41]:
new_text

array([['wheat exhausted in hospital canteen'],
       ['I was abused badly']], dtype=object)

In [42]:
predict_logits

array([1, 4], dtype=int64)

In [43]:
predicts

array([[3.0733538e-01, 6.8997800e-01, 1.0854426e-03, 5.8886816e-04,
        1.0123923e-03],
       [2.7305773e-04, 2.3288429e-03, 1.5826846e-03, 3.0432959e-04,
        9.9551105e-01]], dtype=float32)